In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc as calculate_auc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

from aggmap import AggMap, AggMapNet
from aggmap import show, loadmap

np.random.seed(888) #just for reaptable results

2022-08-16 16:17:41.150563: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
## AggMapNet parameters
epochs = 30 #number of epochs
lr = 1e-4 #learning rate
batch_size = 2 #batch size
conv1_kernel_size = 5 #kernal size of the first cnn layer
gpuid = 1 #use which gpu to train the model

def train(train_country, country_var_thr, n_epochs):
    
    ## data process parameters
    minv = 1e-8 #minimal value for log-transform
    scale_method = 'standard' #data scaling method
    url = 'https://raw.githubusercontent.com/shenwanxiang/bidd-aggmap/master/docs/source/_example_MEGMA/dataset/'
    
    dfx_vector = pd.read_csv(url + '%s_dfx.csv' % train_country, index_col='Sample_ID')
    dfx_vector = np.log(dfx_vector + minv)
    
    megma = AggMap(dfx_vector, by_scipy=True)
    megma = megma.fit(
                      n_epochs = n_epochs,
                      #can be optimized by each megma based on their model performance, such as 1, 2, 4, 8, 10, ..
                      var_thr = country_var_thr,  
                      verbose = 0)
    
    dfy = pd.read_csv(url + '%s_dfy.csv' % train_country, index_col='Sample_ID')

    X = megma.batch_transform(dfx_vector.values, 
                              scale_method = scale_method)
    Y = pd.get_dummies(dfy.Group).values
    
    # fit AggMapNet
    clf = AggMapNet.MultiClassEstimator(epochs = epochs,  
                                        batch_size = batch_size, 
                                        lr = lr, 
                                        conv1_kernel_size = conv1_kernel_size,
                                        gpuid = gpuid,
                                        verbose = 0)
    clf.fit(X, Y)
    return clf, megma

def test(clf, megma, test_country):
    
    ## data process parameters
    minv = 1e-8 #minimal value for log-transform
    scale_method = 'standard' #data scaling method
    url = 'https://raw.githubusercontent.com/shenwanxiang/bidd-aggmap/master/docs/source/_example_MEGMA/dataset/'
    
    dfx_vector = pd.read_csv(url + '%s_dfx.csv' % test_country, index_col='Sample_ID')
    dfx_vector = np.log(dfx_vector + minv)
    dfy = pd.read_csv(url + '%s_dfy.csv' % test_country, index_col='Sample_ID')
    testX = megma.batch_transform(dfx_vector.values, 
                                  scale_method = scale_method)
    testY = pd.get_dummies(dfy.Group).values 
    y_true = testY[:,0]
    y_pred = clf.predict(testX)[:,0] 
    y_score = clf.predict_proba(testX)[:,0]

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    acc = (tp + tn) / sum([tn, fp, fn, tp])
    sensitivity = tp / sum([tp, fn])
    specificity = tn / sum([tn, fp])
    roc_auc = roc_auc_score(y_true, y_score)
    precision = tp / sum([tp, fp])
    recall =  tp / sum([tp, fn]) #equals to sensitivity
    
    res = {'test_country':test_country,
           'accuracy':acc, 
           'roc_auc':roc_auc,
           'sensitivity': sensitivity, 
           'specificity': specificity,
           'precision':precision,
           'recall':recall}
    return res

def get_non_diag_mean(df):
    v = df.values.copy()
    np.fill_diagonal(v, np.nan)
    mean =  pd.DataFrame(v, columns = df.columns, index = df.index).mean(axis=1).round(3)
    std = pd.DataFrame(v, columns = df.columns, index = df.index).std(axis=1).round(3)
    return mean,std

In [3]:
def opt(country_var_thr, n_epochs):
    
    countries = ['AUS', 'CHN', 'DEU', 'FRA', 'USA'] 
    train_country = 'AUS'
    all_res = []
    clf, megma = train(train_country, country_var_thr, n_epochs)
        # Test model performance using STST strategy
    for test_country in countries:
        res = test(clf, megma, test_country)
        res.update({'train_country':train_country})
        all_res.append(res)   
            
    df2 = pd.DataFrame(all_res)
    dfres2 = pd.crosstab(index = df2.train_country, 
                        columns= df2.test_country, 
                        values= df2.roc_auc, 
                        aggfunc = np.mean)
    
    score = get_non_diag_mean(dfres2)[0].mean() #0.808
    
    return score

In [4]:
res = []
for n_epochs in tqdm([50, 100, 200, 300, 500]):
    for var_thr in [2.5, 3.5, 4.5, 5.5, 6.5, 7.5]:
        print(n_epochs, var_thr)
        score = opt(var_thr, n_epochs)
        res.append([var_thr, n_epochs, score])

  0%|                                                                                             | 0/5 [00:00<?, ?it/s]

50 2.5
2022-08-16 16:17:41,972 - INFO - [bidd-aggmap] - Calculating distance ...



100%|###############################################################################| 849/849 [00:00<00:00, 5171.57it/s]

2022-08-16 16:17:42,156 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...


2022-08-16 16:17:44,211 - INFO - [bidd-aggmap] - Applying grid assignment of feature points, this may take several minutes(1~30 min)
2022-08-16 16:17:44,722 - INFO - [bidd-aggmap] - Finished


...

2022-08-16 16:26:57,812 - INFO - [bidd-aggmap] - applying hierarchical clustering to obtain group information ...


2022-08-16 16:26:58,225 - INFO - [bidd-aggmap] - Applying grid assignment of feature points, this may take several minutes(1~30 min)
2022-08-16 16:26:58,454 - INFO - [bidd-aggmap] - Finished



100%|###############################################################################| 109/109 [00:00<00:00, 2598.45it/s]

MultiClassEstimator(batch_norm=False, batch_size=2, conv1_kernel_size=5,
                    dense_avf='relu', dense_layers=[128], dropout=0.0,
                    epochs=30, gpuid='1', last_avf='softmax',
                    loss='categorical_crossentropy', lr=0.0001, metric='ACC',
                    monitor='val_loss', n_inception=2,
                    name='AggMap MultiClass Estimator', patience=10000,
                    random_state=32, verbose=0)




100%|###############################################################################| 109/109 [00:00<00:00, 2491.83it/s]

100%|###############################################################################| 128/128 [00:00<00:00, 2493.75it/s]

100%|###############################################################################| 120/120 [00:00<00:00, 2709.13it/s]

100%|###############################################################################| 114/114 [00:00<00:00, 2530.70it/s]

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [09:33<00:00, 114.65s/it]


In [6]:
pd.DataFrame(res).sort_values(2)

,0,1,2
25,3.5,500,0.64
19,3.5,300,0.64
1,3.5,50,0.65
26,4.5,500,0.66
7,3.5,100,0.66
0,2.5,50,0.67
24,2.5,500,0.67
6,2.5,100,0.67
8,4.5,100,0.67
11,7.5,100,0.67
